In [ ]:
import pandas as pd
from lxml import html
import csv
import aiohttp
import asyncio

In [ ]:
num_pages = 184412
contents = []
async def main():
    async with aiohttp.ClientSession() as session:
        count = 1
        for i in range(1, num_pages):
          async with session.get(f"https://phishtank.org/phish_search.php?page={i}&valid=y&Search=Search") as resp:
            response = await resp.text()
            print(f"{i} : {resp.status}")
            if resp.status == 200:
              contents.append(response)
await main()

In [ ]:
phish_data = []
def extract_data(row):
    phish_dict = {}
    phish_dict["phish_id"] = row.xpath('td')[0].xpath('a/text()')[0]
    phish_dict["phish_url"] = row.xpath('td')[1].xpath('text()')[0]
    phish_dict["date_added"] = row.xpath('td')[1].xpath('span/text()')[0]
    phish_dict["added_by"] = row.xpath('td')[2].xpath('a/text()')[0]
    phish_dict['active'] = row.xpath('td')[4].xpath('.//text()')[0]
    phish_data.append(phish_dict)

In [ ]:
for i in contents:
    tree = html.fromstring(i)
    rows = tree.xpath('//tr[@style="background: #ffffcc;"]')
    for j in rows:
        extract_data(j)

In [ ]:
df = pd.DataFrame(phish_data)
df.to_csv('PhishTank_Data2.csv', index=False, quoting=csv.QUOTE_ALL, encoding='utf-8')